# ROI-based univariate analyses
Natalia Vélez, April 2022

In [1]:
%matplotlib inline

import os, sys
import pandas as pd
import numpy as np
from nilearn import image,plotting,masking
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from os.path import join as opj
from scipy.io import loadmat
from scipy import stats

sys.path.append('..')
from utils import gsearch, str_extract, print_list

sns.set_style('white')
sns.set_context('talk')

ModuleNotFoundError: No module named 'nilearn'

## Setup

Valid subjects:

In [ ]:
subjects = np.loadtxt('../1_preprocessing/outputs/valid_participants.txt', dtype=int)
print(subjects)

Project directory:

In [ ]:
data_dir = '/n/gershman_ncf/Lab/natalia_teaching/BIDS_data/derivatives'
model_name = 'task-teaching_model-parametric'

Find ACC:

In [ ]:
acc_file = '/ncf/gershman/User/nvelezalicea/fmri_analysis/roi_library/fmriprep_space/bilateral_ACCg.nii.gz'
os.path.exists(acc_file)

Find functional ROI files:

In [ ]:
roi_files = gsearch(data_dir, 'roi_picker', 'sub-*', 'func', '*.nii.gz')
roi_files.sort()

print_list(roi_files, 'ROI files')

Find contrast files:

In [ ]:
con_files = gsearch(data_dir, 'glm', 'sub-*', 'func', model_name, 'con*')
con_files.sort()

print_list(con_files, 'contrast files')

Load contrast names:

In [ ]:
con_file = opj(data_dir, 'glm', 'group', model_name, 'contrasts.mat')
con_data = loadmat(con_file)
contrasts_raw = con_data['contrasts']
contrasts = [c[0].replace('+', '') for c in contrasts_raw[0]] # all a hacky way of reading mat files

print_list(contrasts, 'contrasts')

## GLM 1: Full model with model-based regressors

### First-level contrasts

Main loop: Extract data from all ROIs

In [ ]:
# Initialize output
roi_list = []

# Iterate over subjects and contrasts
for sub in subjects:
    for con_idx,con_name in enumerate(contrasts):
        
        # Format subject, contrast numbers nicely
        sub_id = 'sub-%02d' % sub
        con_id = 'con_%04d' % (con_idx+1)

        # Filter out contrast, ROI files
        sub_con = [f for f in con_files if sub_id in f and con_id in f]
        sub_con = sub_con[0] # should be a unique file

        sub_roi = [f for f in roi_files if sub_id in f]
        sub_roi.append(acc_file) # add anatomical ACC ROI
        sub_roi.sort() # list of files
                
        # Extract average t from each ROI
        for roi in sub_roi:
            roi_name = str_extract('(?<=desc-)[A-Za-z]+|ACC', roi)            
            masked_t = masking.apply_mask(sub_con, roi)
            mean_t = masked_t.mean()
            
            roi_list.append((sub, con_name, roi_name, mean_t))

In [ ]:
# put it all together
roi_df = pd.DataFrame(roi_list, columns=['subject', 'contrast', 'roi', 'beta'])
roi_df['roi'] = roi_df.roi.astype('category')
roi_df = roi_df.sort_values(by=['subject', 'contrast', 'roi']).reset_index(drop=True)
print(roi_df.shape)
roi_df.head(8)

Do one-sample t-tests with Bonferroni correction:

In [ ]:
t_list = []

# do one-sample t-tests on average ROI betas
for name,group in roi_df.groupby(['roi', 'contrast']):
    # find CI (for plotting)
    ci_lo, ci_hi = sns.utils.ci(sns.algorithms.bootstrap(group.beta))
    group_mean = group.beta.mean()
    
    res = stats.ttest_1samp(group.beta, 0)
    t_list.append(name+(group_mean, res.statistic, res.pvalue, ci_lo, ci_hi))
    
t_df = (
    pd.DataFrame(t_list, columns=['roi', 'contrast', 'avg', 'statistic', 'pvalue', 'ci_lo', 'ci_hi'])
    .sort_values(by=['contrast', 'roi'])
    .reset_index(drop=True)
)
t_df['roi'] = t_df['roi'].astype('category')

# mark ROIs that survive Bonferroni correction
p_thresh = 0.05/(t_df.roi.nunique())
t_df['sig'] = np.where(t_df.pvalue < p_thresh, '*', '') # extra vars for plotting
t_df['sig_y'] = np.where(t_df.avg >= 0, t_df.ci_hi+.1, t_df.ci_lo-.1)
t_df['sig_x'] = t_df.roi.cat.codes-.11

t_df

#### 1) Posterior probability of true hypothesis

Plot results:

In [ ]:
roi_colors = ['#1A7AB4', '#D79316', '#18A27B', '#CB7FBD', '#C9976B', '#9C9C9C', '#E5DD40', '#62B3E4']

pTrue_df = roi_df[roi_df.contrast == 'pTrue']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=pTrue_df, x='roi', y='beta', ax=ax, palette=roi_colors, saturation=1)

pTrue_sig = t_df[t_df.contrast == 'pTrue']
for _, row in pTrue_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = r'Belief in correct answer: $P_L(h_T|d)$', ylim=(
    pTrue_sig.ci_lo.min()*1.25,
    pTrue_sig.ci_hi.max()*1.25
))

plt.savefig('plots/main_pTrue.png')

#### 2) Belief update

In [ ]:
KL_df = roi_df[roi_df.contrast == 'KL']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=KL_df, x='roi', y='beta', fc='#ff9b0f', ax=ax)

KL_sig = t_df[t_df.contrast == 'KL']
for _, row in KL_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = 'Belief update (KL divergence)', ylim=(
    KL_sig.ci_lo.min()*1.25,
    KL_sig.ci_hi.max()*1.25
))

plt.savefig('plots/main_KL.png')

## GLM 2: Parametric regressors derived from student responses

Empirical regressor files:

In [ ]:
empirical_con = gsearch(data_dir, 'glm', 'sub-*', 'func', 'task-teaching_model-student', 'con*')
empirical_con.sort()

print_list(empirical_con, 'contrast files')

Extract average betas from ROIs:

In [ ]:
# Initialize output
empirical_list = []

# Iterate over subjects and contrasts
for sub in subjects:
    for con_idx,con_name in enumerate(contrasts):
        
        # Format subject, contrast numbers nicely
        sub_id = 'sub-%02d' % sub
        con_id = 'con_%04d' % (con_idx+1)

        # Filter out contrast, ROI files
        sub_con = [f for f in empirical_con if sub_id in f and con_id in f]
        sub_con = sub_con[0] # should be a unique file

        sub_roi = [f for f in roi_files if sub_id in f]
        sub_roi.append(acc_file) # add anatomical ACC ROI
        sub_roi.sort() # list of files
                
        # Extract average t from each ROI
        for roi in sub_roi:
            roi_name = str_extract('(?<=desc-)[A-Za-z]+|ACC', roi)            
            masked_t = masking.apply_mask(sub_con, roi)
            mean_t = masked_t.mean()
            
            empirical_list.append((sub, con_name, roi_name, mean_t))

In [ ]:
# put it all together
empirical_df = pd.DataFrame(empirical_list, columns=['subject', 'contrast', 'roi', 'beta'])
empirical_df['roi'] = empirical_df.roi.astype('category')
empirical_df = empirical_df.sort_values(by=['subject', 'contrast', 'roi']).reset_index(drop=True)
print(empirical_df.shape)
empirical_df.head(8)

Do one-sample t-tests with Bonferroni correction

In [ ]:
t_list = []

# do one-sample t-tests on average ROI betas
for name,group in empirical_df.groupby(['roi', 'contrast']):
    # find CI (for plotting)
    ci_lo, ci_hi = sns.utils.ci(sns.algorithms.bootstrap(group.beta))
    group_mean = group.beta.mean()
    
    res = stats.ttest_1samp(group.beta, 0)
    t_list.append(name+(group_mean, res.statistic, res.pvalue, ci_lo, ci_hi))
    
empirical_t = (
    pd.DataFrame(t_list, columns=['roi', 'contrast', 'avg', 'statistic', 'pvalue', 'ci_lo', 'ci_hi'])
    .sort_values(by=['contrast', 'roi'])
    .reset_index(drop=True)
)
empirical_t['roi'] = empirical_t['roi'].astype('category')

# mark ROIs that survive Bonferroni correction
p_thresh = 0.05/(empirical_t.roi.nunique())
empirical_t['sig'] = np.where(empirical_t.pvalue < p_thresh, '*', '') # extra vars for plotting
empirical_t['sig_y'] = np.where(empirical_t.avg >= 0, empirical_t.ci_hi+.1, empirical_t.ci_lo-.3)
empirical_t['sig_x'] = empirical_t.roi.cat.codes-.11

empirical_t

Plot results:

In [ ]:
pTrue_df = empirical_df[empirical_df.contrast == 'pTrue']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=pTrue_df, x='roi', y='beta', ax=ax, palette=roi_colors, saturation=1)

pTrue_sig = empirical_t[empirical_t.contrast == 'pTrue']
for _, row in pTrue_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = r'Belief in correct answer: $P_L(h_T|d)$', ylim=(
    pTrue_sig.ci_lo.min()*1.25,
    pTrue_sig.ci_hi.max()*1.25
))

plt.savefig('plots/empirical_pTrue.png')

In [ ]:
KL_df = empirical_df[empirical_df.contrast == 'KL']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=KL_df, x='roi', y='beta', fc='#ff9b0f', ax=ax)

KL_sig = empirical_t[empirical_t.contrast == 'KL']
for _, row in KL_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = 'Belief update (KL divergence)', ylim=(
    KL_sig.ci_lo.min()*1.25,
    KL_sig.ci_hi.max()*1.25
))

plt.savefig('plots/empirical_KL.png')

## GLM 3-4: Model-based regressors estimated in separate GLMs

#### pTrue

Regressor files:

In [ ]:
pTrue_con = gsearch(data_dir, 'glm', 'sub-*', 'func', 'task-teaching_model-parametricpTrue', 'con_0001*')
pTrue_con.sort()

print_list(pTrue_con, 'contrast files')

Extract average betas from ROIs:

In [ ]:
# Initialize output
pTrue_list = []

# Iterate over subjects and contrasts
for sub in subjects:
    sub_id = 'sub-%02d' % sub
    con_name = 'pTrue'
    # Filter out contrast, ROI files
    sub_con = [f for f in pTrue_con if sub_id in f]
    sub_con = sub_con[0] # should be a unique file

    sub_roi = [f for f in roi_files if sub_id in f]
    sub_roi.append(acc_file) # add anatomical ACC ROI
    sub_roi.sort() # list of files

    # Extract average t from each ROI
    for roi in sub_roi:
        roi_name = str_extract('(?<=desc-)[A-Za-z]+|ACC', roi)            
        masked_t = masking.apply_mask(sub_con, roi)
        mean_t = masked_t.mean()

        pTrue_list.append((sub, con_name, roi_name, mean_t))

In [ ]:
# put it all together
pTrue_df = pd.DataFrame(pTrue_list, columns=['subject', 'contrast', 'roi', 'beta'])
pTrue_df['roi'] = pTrue_df.roi.astype('category')
pTrue_df = pTrue_df.sort_values(by=['subject', 'contrast', 'roi']).reset_index(drop=True)
print(pTrue_df.shape)
pTrue_df.head(8)

Do one-sample t-tests with Bonferroni correction

In [ ]:
pTrue_t_list = []

# do one-sample t-tests on average ROI betas
for name,group in pTrue_df.groupby(['roi', 'contrast']):
    # find CI (for plotting)
    ci_lo, ci_hi = sns.utils.ci(sns.algorithms.bootstrap(group.beta))
    group_mean = group.beta.mean()
    
    res = stats.ttest_1samp(group.beta, 0)
    pTrue_t_list.append(name+(group_mean, res.statistic, res.pvalue, ci_lo, ci_hi))
    
pTrue_t = (
    pd.DataFrame(pTrue_t_list, columns=['roi', 'contrast', 'avg', 'statistic', 'pvalue', 'ci_lo', 'ci_hi'])
    .sort_values(by=['contrast', 'roi'])
    .reset_index(drop=True)
)
pTrue_t['roi'] = pTrue_t['roi'].astype('category')

# mark ROIs that survive Bonferroni correction
p_thresh = 0.05/(pTrue_t.roi.nunique())
pTrue_t['sig'] = np.where(pTrue_t.pvalue < p_thresh, '*', '') # extra vars for plotting
pTrue_t['sig_y'] = np.where(pTrue_t.avg >= 0, pTrue_t.ci_hi+.1, pTrue_t.ci_lo-.3)
pTrue_t['sig_x'] = pTrue_t.roi.cat.codes-.11

pTrue_t

Plot results:

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=pTrue_df, x='roi', y='beta', fc='#007fcf', ax=ax)

for _, row in pTrue_t.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = r'Belief in true hypothesis: $P_L(h_T|d)$', ylim=(
    pTrue_sig.ci_lo.min()*1.25,
    pTrue_sig.ci_hi.max()*1.25
))

plt.savefig('plots/separate_pTrue.png')

#### KL

Regressor files:

In [ ]:
KL_con = gsearch(data_dir, 'glm', 'sub-*', 'func', 'task-teaching_model-parametricKL', 'con_0001*')
KL_con.sort()

print_list(KL_con, 'contrast files')

Extract average betas from ROIs:

In [ ]:
# Initialize output
KL_list = []

# Iterate over subjects and contrasts
for sub in subjects:
    sub_id = 'sub-%02d' % sub
    con_name = 'pTrue'
    # Filter out contrast, ROI files
    sub_con = [f for f in KL_con if sub_id in f]
    sub_con = sub_con[0] # should be a unique file

    sub_roi = [f for f in roi_files if sub_id in f]
    sub_roi.append(acc_file) # add anatomical ACC ROI
    sub_roi.sort() # list of files

    # Extract average t from each ROI
    for roi in sub_roi:
        roi_name = str_extract('(?<=desc-)[A-Za-z]+|ACC', roi)            
        masked_t = masking.apply_mask(sub_con, roi)
        mean_t = masked_t.mean()

        KL_list.append((sub, con_name, roi_name, mean_t))

In [ ]:
# put it all together
KL_df = pd.DataFrame(KL_list, columns=['subject', 'contrast', 'roi', 'beta'])
KL_df['roi'] = KL_df.roi.astype('category')
KL_df = KL_df.sort_values(by=['subject', 'contrast', 'roi']).reset_index(drop=True)
print(KL_df.shape)
KL_df.head(8)

Do one-sample t-tests with Bonferroni correction

In [ ]:
KL_t_list = []

# do one-sample t-tests on average ROI betas
for name,group in KL_df.groupby(['roi', 'contrast']):
    # find CI (for plotting)
    ci_lo, ci_hi = sns.utils.ci(sns.algorithms.bootstrap(group.beta))
    group_mean = group.beta.mean()
    
    res = stats.ttest_1samp(group.beta, 0)
    KL_t_list.append(name+(group_mean, res.statistic, res.pvalue, ci_lo, ci_hi))
    
KL_t = (
    pd.DataFrame(KL_t_list, columns=['roi', 'contrast', 'avg', 'statistic', 'pvalue', 'ci_lo', 'ci_hi'])
    .sort_values(by=['contrast', 'roi'])
    .reset_index(drop=True)
)
KL_t['roi'] = KL_t['roi'].astype('category')

# mark ROIs that survive Bonferroni correction
p_thresh = 0.05/(KL_t.roi.nunique())
KL_t['sig'] = np.where(KL_t.pvalue < p_thresh, '*', '') # extra vars for plotting
KL_t['sig_y'] = np.where(KL_t.avg >= 0, KL_t.ci_hi+.1, KL_t.ci_lo-.1)
KL_t['sig_x'] = KL_t.roi.cat.codes-.11

KL_t

Plot results:

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=KL_df, x='roi', y='beta', fc='#ff9b0f', ax=ax)

for _, row in KL_t.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = r'Belief update (KL divergence)', ylim=(
    KL_t.ci_lo.min()*1.25,
    KL_t.ci_hi.max()*1.25
))

plt.savefig('plots/separate_KL.png')

In [ ]:
KL_df = empirical_df[empirical_df.contrast == 'KL']
fig,ax = plt.subplots(figsize=(8,4))
sns.barplot(data=KL_df, x='roi', y='beta', fc='#ff9b0f', ax=ax)

KL_sig = empirical_t[empirical_t.contrast == 'KL']
for _, row in KL_sig.iterrows():
    ax.text(row.sig_x, row.sig_y, row.sig)

ax.set(xlabel='', ylabel='Beta', title = 'Belief update (KL divergence)', ylim=(
    KL_sig.ci_lo.min()*1.25,
    KL_sig.ci_hi.max()*1.25
))